In [4]:
import os
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
)

In [5]:
DEMO_PATH = "notebook/integration_demo/"
ASKENET_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"

## load_and_sample_petri_model

In [6]:
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run sampling
samples = load_and_sample_petri_model(ASKENET_PATH, num_samples, timepoints=timepoints)

# Save results
samples.to_csv(os.path.join(DEMO_PATH, "results_petri/sample_results.csv"), index=False)

## load_and_calibrate_and_sample_petri_model

In [7]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_petri_model(
    ASKENET_PATH,
    data_path,
    num_samples,
    timepoints=timepoints,
    verbose=True,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/calibrated_sample_results.csv"), index=False
)

iteration 0: loss = 37.94811773300171
iteration 25: loss = 34.9170196056366
iteration 50: loss = 32.69080424308777
iteration 75: loss = 32.41730046272278
iteration 100: loss = 32.44331431388855
iteration 125: loss = 32.692076683044434
iteration 150: loss = 32.61838483810425
iteration 175: loss = 32.56607246398926
iteration 200: loss = 33.014737367630005
iteration 225: loss = 32.58268332481384
iteration 250: loss = 32.89216685295105
iteration 275: loss = 31.961626768112183
iteration 300: loss = 32.6565682888031
